## ML Procedure
#### 1. Read in and clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [4]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']
data.head()

,label,body_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,"Nah I don't think he goes to usf, he lives aro..."
2,ham,Even my brother is not like to speak with me. ...
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
4,ham,As per your request 'Melle Melle (Oru Minnamin...


In [5]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

In [7]:
data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))
data.head()

,label,body_text,body_len,punct%
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,128,4.7
1,ham,"Nah I don't think he goes to usf, he lives aro...",49,4.1
2,ham,Even my brother is not like to speak with me. ...,62,3.2
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,28,7.1
4,ham,As per your request 'Melle Melle (Oru Minnamin...,135,4.4


In [8]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [11]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf

<5567x8104 sparse matrix of type '<class 'numpy.float64'>'
	with 50122 stored elements in Compressed Sparse Row format>

In [12]:
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis = 1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 2. Explore RandomForestClassifier & Run through Cross-Validation

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_compute_oob_predictions', '_estimator_type', '_get_oob_predictions', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score_and_attributes', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'n_features_', 'predict', 'predict_log_proba', 'predict_proba', 'score',

In [15]:
from sklearn.model_selection import KFold, cross_val_score

In [16]:
rf = RandomForestClassifier(n_jobs=-1) #allows it to run faster by building in parallel
k_fold = KFold(n_splits=5) #uses five subset cross validation, training on first four eval on 5, and so on

cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1) #needs to know model (rf), input features, label(orig dataset), splits (k_fold), scoring metric (accuracy or precision), jobs in parallel

array([0.97486535, 0.97576302, 0.97394429, 0.96495957, 0.97304582])

#### 3. Build Random Forest on holdout test set

In [17]:
from sklearn.metrics import precision_recall_fscore_support as score  
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2) #splitting and identifying what we want data outputted to

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1) #n_estimators (default, 10), max_depth = depth of each tree - default = none.
rf_model = rf.fit(X_train, y_train) #fit model with fit attribute and pass in the x and y training data

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [21]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10] #printing the first ten rows of feature importance, most important first. Wrap with zip to take to arrays and combine

[(0.06714150263277707, 'body_len'),
 (0.027723685240798907, 1803),
 (0.02577571118823116, 7461),
 (0.025719259397338906, 5724),
 (0.02369094015790017, 7350),
 (0.022434789327533382, 7027),
 (0.02131516865007976, 3134),
 (0.02092879105791857, 2031),
 (0.020775040670838008, 4796),
 (0.017687728477333253, 6746)]

In [22]:
y_pred = rf_model.predict(X_test) #.predict operator from the fit model and pass in the test data (only x_test - prediction based on x features)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary') #have to declare positive label if label is not binary (what we are interested in predicting)

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [24]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))
#When model identified spam, it was spam 100% of the time. Spam into email, 55% was properly placed in spam folder, 94% correctly identified as spam/not spam - model not quite aggressive enough to prevent making it to inbox

Precision: 1.0 / Recall: 0.551 / Accuracy: 0.945


#### 4. Explore RF model with grid-search

In [25]:
#Building grid search
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [26]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [27]:
#Adding depth creating lots of improvement

for n_est in [10, 50, 100]:
    for depth in [10, 20, 30]:
        train_RF(n_est, depth)

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.347 / Accuracy 0.912


c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.547 / Accuracy 0.939


c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


...

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.193 / Accuracy 0.891


c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.587 / Accuracy 0.944
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.673 / Accuracy 0.956


c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [28]:
# All these future warnings are making me feel old :(

#### 5. Evaluating RF with GridsearchCV

In [29]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

In [30]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

#TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis = 1)

#Countvectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis = 1)

X_count_feat.head()


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300], 'max_depth': [30, 60, 90, None]}

GS = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = GS.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,22.679643,0.733733,0.322356,0.049233,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.980251,0.978456,0.974843,0.968553,0.972147,0.974850,0.004222,1
8,44.001387,2.492658,0.519008,0.165671,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.978456,0.975741,0.966757,0.973944,0.974491,0.004167,2
11,37.935873,4.362807,0.307511,0.031660,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.978456,0.973944,0.968553,0.972147,0.974311,0.003803,3
10,25.516905,1.332226,0.384245,0.103413,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.976661,0.974843,0.967655,0.971249,0.973593,0.003673,4
4,19.519663,1.218479,0.281860,0.017730,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.976661,0.973070,0.973046,0.968553,0.973046,0.972875,0.002573,5


In [33]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300], 'max_depth': [30, 60, 90, None]}

GS = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = GS.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,34.702440,4.132253,0.293439,0.017067,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975763,0.974865,0.974843,0.968553,0.971249,0.973055,0.002733,1
7,21.030319,0.922319,0.348679,0.044245,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.974865,0.972147,0.968553,0.971249,0.972875,0.003090,2
8,41.275378,2.860800,0.438619,0.147668,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.974865,0.973046,0.965858,0.970350,0.972336,0.004002,3
10,24.123205,1.748443,0.362787,0.079343,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.974865,0.973944,0.966757,0.968553,0.972156,0.003820,4
3,2.881722,0.384024,0.215631,0.044726,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.973070,0.974865,0.969452,0.964061,0.973046,0.970899,0.003845,5


#### 6. Gradient Boosting

In [38]:
#Evaluation based on error minimization through iterative trials - more powerful and better performing IF tuned properly
#Accepts various types of inputs, use for classificaiton or regression
#Note: max depth default = 3
#Note: no n_jobs param
#Note: learning rate param

In [35]:
from sklearn.ensemble import GradientBoostingClassifier

In [36]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_initialized', '_check_n_features', '_check_params', '_clear_state', '_compute_partial_dependence_recursion', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_more_tags', '_raw_predict', '_raw_predict_init', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_data', '_validate_estimator', '_validate_y', '_warn_ma

In [39]:
#Lots of very basic trees, as oppsed to rf, which is a bunch of big trees.
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [42]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ------ Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [43]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feat

Est: 50 / Depth: 3 / LR: 0.01 ------ Precision: 0.0 / Recall: 0.0 / Accuracy: 0.865


c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Est: 50 / Depth: 3 / LR: 0.1 ------ Precision: 0.955 / Recall: 0.707 / Accuracy: 0.956


c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


...

c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Est: 100 / Depth: 15 / LR: 0.01 ------ Precision: 0.966 / Recall: 0.76 / Accuracy: 0.964


c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\rymer\anaconda3\envs\VS_Code\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Est: 100 / Depth: 15 / LR: 0.1 ------ Precision: 0.939 / Recall: 0.827 / Accuracy: 0.969


: 